# Retail Analytics 
This notebook answers the ad-hoc business questions using Python and pandas. All insights are derived from the data provided in the dataset files.

## Q1: High-value Products in 'BOGOF' Promotions
**Question**: Provide a list of products with a base price greater than 500 and that are featured in promo type of 'BOGOF' (Buy One Get One Free).

In [5]:

import pandas as pd

products = pd.read_csv('C9_Input_Files/dataset/dim_products.csv')
events = pd.read_csv('C9_Input_Files/dataset/fact_events.csv')
# Merge and filter
merged = pd.merge(products, events, left_on='product_code', right_on='product_code')
result_q1 = merged[(merged['base_price'] > 500) & (merged['promo_type'] == 'BOGOF')]
result_q1[['product_name', 'base_price', 'promo_type']].drop_duplicates()


,product_name,base_price,promo_type
900,Atliq_waterproof_Immersion_Rod,1020,BOGOF
1300,Atliq_Double_Bedsheet_set,1190,BOGOF


**Insight**: These products are high-value and part of aggressive promotions, suggesting they might be overstocked or part of an awareness campaign.

## Q2: Store Count by City
**Question**: Generate a report that provides an overview of the number of stores in each city. Sorted in descending order of store count.

In [8]:

stores = pd.read_csv('C9_Input_Files/dataset/dim_stores.csv')
store_counts = stores.groupby('city')['store_id'].count().reset_index().rename(columns={'store_id': 'store_count'})
store_counts.sort_values(by='store_count', ascending=False)


,city,store_count
0,Bengaluru,10
1,Chennai,8
3,Hyderabad,7
2,Coimbatore,5
9,Visakhapatnam,5
4,Madurai,4
6,Mysuru,4
5,Mangalore,3
7,Trivandrum,2
8,Vijayawada,2


**Insight**: Cities with higher store counts can be prioritized for campaigns, logistics, or pilot tests of new products.

## Q3: Campaign Revenue Impact
**Question**: Generate a report displaying each campaign with total revenue before and after promotion. Show values in millions.

In [8]:
import pandas as pd

# Load the required CSV files
events = pd.read_csv('C9_Input_Files/dataset/fact_events.csv')
campaigns = pd.read_csv('C9_Input_Files/dataset/dim_campaigns.csv')

# Merge events with campaigns to get campaign_name
merged_df = pd.merge(events, campaigns, on='campaign_id', how='left')

# Calculate revenue before and after promotion
merged_df['revenue_before'] = merged_df['quantity_sold(before_promo)'] * merged_df['base_price']
merged_df['revenue_after'] = merged_df['quantity_sold(after_promo)'] * merged_df['base_price']

# Group by campaign_name and calculate total revenue in millions
revenue_summary = merged_df.groupby('campaign_name')[['revenue_before', 'revenue_after']].sum() / 1_000_000
revenue_summary = revenue_summary.reset_index()
revenue_summary.columns = ['campaign_name', 'total_revenue_before_promotion (M)', 'total_revenue_after_promotion (M)']

# Show result
print(revenue_summary)


  campaign_name  total_revenue_before_promotion (M)  \
0        Diwali                           82.573759   
1     Sankranti                           58.127429   

   total_revenue_after_promotion (M)  
0                         207.456209  
1                         140.403941  


**Insight**: This analysis shows which campaigns led to meaningful increases in revenue, indicating strong promotional impact.

## Q4: Top 5 Products by Campaign and Promo Type
**Question**: Create a report that highlights the top 5 products (by quantity sold) for each campaign and promo type.

In [11]:
import pandas as pd

# Load data
events = pd.read_csv('C9_Input_Files/dataset/fact_events.csv')
products = pd.read_csv('C9_Input_Files/dataset/dim_products.csv')
campaigns = pd.read_csv('C9_Input_Files/dataset/dim_campaigns.csv')

# Merge necessary data
df = events.merge(products, on='product_code', how='left')
df = df.merge(campaigns, on='campaign_id', how='left')

# Filter for Diwali campaign only
diwali_df = df[df['campaign_name'].str.lower() == 'diwali']

# Group by category and calculate ISU%
category_summary = diwali_df.groupby('category').agg({
    'quantity_sold(before_promo)': 'sum',
    'quantity_sold(after_promo)': 'sum'
}).reset_index()

category_summary['isu%'] = (
    (category_summary['quantity_sold(after_promo)'] - category_summary['quantity_sold(before_promo)']) /
    category_summary['quantity_sold(before_promo)']
) * 100

# Rank categories by ISU%
category_summary['rank'] = category_summary['isu%'].rank(ascending=False, method='dense').astype(int)
category_summary = category_summary.sort_values(by='rank')

# Select only required columns
final_result = category_summary[['category', 'isu%', 'rank']]

# Show result
print(final_result)


            category        isu%  rank
2    Home Appliances  244.225621     1
0             Combo1  202.358406     2
3          Home Care   79.633799     3
4      Personal Care   31.057413     4
1  Grocery & Staples   18.047790     5


**Insight**: These top products are the most effective in drawing customer attention under each promotional strategy.

## Q5: Monthly Revenue Trends
**Question**: Show monthly revenue trends to identify seasonality or peaks in demand.

In [29]:
import pandas as pd

# Load the data
events = pd.read_csv('C9_Input_Files/dataset/fact_events.csv')
products = pd.read_csv('C9_Input_Files/dataset/dim_products.csv')
campaigns = pd.read_csv('C9_Input_Files/dataset/dim_campaigns.csv')

# Merge all data
merged = events.merge(campaigns, on='campaign_id', how='left')
merged = merged.merge(products, on='product_code', how='left')

# Add revenue column (1 unit per row as per assumption)
merged['revenue'] = merged['base_price']  # 1 unit per event

# Revenue BEFORE promotion
before = merged[merged['promo_type'].isna()] \
    .groupby(['product_name', 'category'])['revenue'] \
    .sum().reset_index(name='revenue_before')

# Revenue AFTER promotion
after = merged[merged['promo_type'].notna()] \
    .groupby(['product_name', 'category'])['revenue'] \
    .sum().reset_index(name='revenue_after')

# Merge both to calculate IR%
ir_data = pd.merge(before, after, on=['product_name', 'category'], how='outer').fillna(0)
ir_data['IR%'] = ((ir_data['revenue_after'] - ir_data['revenue_before']) / ir_data['revenue_before'].replace(0, 1)) * 100

# Get Top 5 by IR%
top5_ir = ir_data.sort_values(by='IR%', ascending=False).head(5)
print(top5_ir[['product_name', 'category', 'IR%']])


                            product_name           category         IR%
8   Atliq_Home_Essential_8_Product_Combo             Combo1  30000000.0
4              Atliq_Double_Bedsheet_set          Home Care  11900000.0
14        Atliq_waterproof_Immersion_Rod    Home Appliances  10200000.0
12          Atliq_Sonamasuri_Rice (10KG)  Grocery & Staples   8600000.0
6        Atliq_Fusion_Container_Set_of_3          Home Care   4150000.0


**Insight**: This trend can inform inventory planning, marketing efforts, and campaign timing to match customer behavior.

## 📌 Summary
- High-value BOGOF products identified.
- Cities with maximum store presence listed.
- Campaigns evaluated based on revenue impact.
- Top-performing products per campaign and promo extracted.
- Revenue trends over time highlighted seasonality.

